# Loading CSV Data into SQL Server
This notebook demonstrates how to load data from CSV files into a SQL Server database using Python, `pyodbc`, and `pandas`. The process includes establishing a connection, creating tables, and inserting data.

## Step 1: Importing Required Libraries
Import necessary libraries for connecting to SQL Server (`pyodbc`), handling file operations (`os`), and working with data (`pandas`).

In [ ]:
import pyodbc
import os
import pandas as pd

## Step 2: Setting Up the Connection String
Define the connection string parameters to connect to the SQL Server database. Make sure to replace the server and database names with your actual values.

In [2]:
server = 'DESKTOP-GJR311M\\PROJECT_BI_2BA2'  # Your server name
database = 'Youtube_DB'  # Replace with your actual database name

# Use Trusted_Connection=yes for Windows Authentication
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
# Test the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connection successful!")
    conn.close()
except Exception as e:
    print(f"Error: {e}")

Connection successful!


## Step 3: Loading channels and their performance CSV Data into SQL Server
This step automates the process of loading data from multiple CSV files into a SQL Server database:

1. **Connect to SQL Server**: Establish a connection using the provided connection string and create a cursor for executing SQL commands.
2. **Folder Path Setup**: Specify the folder containing the CSV files to be imported.
3. **Loop Through CSV Files**: Iterate over each CSV file in the specified folder:
   - **Load the CSV into a DataFrame**: Read the file into a pandas DataFrame.
   - **Create Table in SQL Server**: Generate a `CREATE TABLE` SQL query based on the DataFrame columns, using `NVARCHAR(MAX)` as the data type for each column. The table name is derived from the CSV file name.
   - **Insert Data into the Table**: Insert the data row by row from the DataFrame into the newly created SQL table.
4. **Commit Changes**: Commit the transaction after all rows from the CSV have been inserted to ensure data is saved.
5. **Close the Connection**: Close the SQL Server connection after processing all files.

This approach ensures that each CSV file is loaded into a separate table within the database.


In [15]:
# Connect to SQL Server
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Folder path containing CSV files
folder_path = r'C:\Users\khalil\Desktop\Youtube_Data'

# Loop through each CSV file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        # Full file path
        file_path = os.path.join(folder_path, file_name)
        
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)
        
        # Extract the table name (same as the file name without the .csv extension)
        table_name = os.path.splitext(file_name)[0]
        
        # Prepare the CREATE TABLE SQL query
        create_table_query = f"CREATE TABLE [{table_name}] ("
        for col in df.columns:
            # Enclose column names in square brackets and use NVARCHAR(MAX)
            create_table_query += f"[{col}] NVARCHAR(MAX),"
        create_table_query = create_table_query.rstrip(",") + ")"
        
        # Execute the create table query (this creates a table if it doesn't exist)
        cursor.execute(create_table_query)
        conn.commit()
        
        # Insert DataFrame into SQL Server table row by row
        for index, row in df.iterrows():
            insert_query = f"INSERT INTO [{table_name}] VALUES ({','.join(['?' for _ in row])})"
            cursor.execute(insert_query, tuple(row))
        
        # Commit after inserting all rows for this CSV
        conn.commit()

        print(f"Data from {file_name} inserted into table [{table_name}].")

# Close the connection
conn.close()

Data from all_performance_data_Australia.csv inserted into table [all_performance_data_Australia].
Data from all_performance_data_Canada.csv inserted into table [all_performance_data_Canada].
Data from all_performance_data_UK.csv inserted into table [all_performance_data_UK].
Data from all_performance_data_USA.csv inserted into table [all_performance_data_USA].
Data from CChannels_Australia.csv inserted into table [CChannels_Australia].
Data from Channels_Canada.csv inserted into table [Channels_Canada].
Data from Channels_UK.csv inserted into table [Channels_UK].
Data from Channels_USA.csv inserted into table [Channels_USA].


## Step 4: Loading YouTube Trending Data into SQL Server
This step involves loading the YouTube trending data from a CSV file into a SQL Server database:

1. **Connect to SQL Server**: Establish a connection using the provided connection string and create a cursor for executing SQL commands.
2. **Load CSV Data into a DataFrame**: Read the `youtube_trending_data.csv` file into a pandas DataFrame.
3. **Table Preparation**:
   - **Define the Table Name**: Set the table name as 'trending'.
   - **Drop Existing Table**: If a table named 'trending' already exists in the database, drop it to avoid conflicts.
   - **Create a New Table**: Dynamically generate a `CREATE TABLE` SQL query based on the DataFrame columns, using `NVARCHAR(MAX)` as the data type for each column.
4. **Data Cleaning**:
   - **Define a Cleaning Function**: Use the `clean_value` function to handle invalid or infinite values by converting them to `None`.
5. **Insert Cleaned Data into the Table**:
   - **Iterate Over Rows**: Clean each row using the `clean_value` function and insert the data into the 'trending' table row by row.
6. **Commit Changes**: Commit the transaction after all rows have been inserted to ensure the data is saved.
7. **Close the Connection**: Close the SQL Server connection after completing the data insertion.

This process ensures that the YouTube trending data is loaded into the database in a clean and structured format for further analysis.


In [6]:
# Connect to SQL Server
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Full path to the CSV file
file_path = r'C:\Users\khalil\marketing_data_Analytics_Project\youtube_trending_data.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Extract the table name (for this case, we'll use 'trending')
table_name = 'trending'

# Drop the table if it already exists (this avoids the "table already exists" error)
drop_table_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE [{table_name}]"
cursor.execute(drop_table_query)
conn.commit()

# Prepare the CREATE TABLE SQL query dynamically based on the DataFrame columns
create_table_query = f"CREATE TABLE [{table_name}] ("
for col in df.columns:
    # Enclose column names in square brackets and use NVARCHAR(MAX) as the column type
    create_table_query += f"[{col}] NVARCHAR(MAX),"
create_table_query = create_table_query.rstrip(",") + ")"

# Execute the create table query (creates the table based on the CSV columns)
try:
    cursor.execute(create_table_query)
    conn.commit()
    print(f"Table '{table_name}' created successfully.")
except Exception as e:
    print(f"Table creation failed: {e}")

# Clean the DataFrame before inserting it into SQL Server
def clean_value(value):
    """Convert values to None if they cannot be converted to SQL-appropriate data types."""
    try:
        # Convert infinite values or invalid values to None
        if pd.isna(value) or value == float('inf') or value == float('-inf'):
            return None
        else:
            return str(value)  # Convert to string to avoid SQL data type issues
    except Exception:
        return None

# Insert DataFrame into SQL Server table row by row
for index, row in df.iterrows():
    # Clean each row by applying the clean_value function
    cleaned_row = [clean_value(val) for val in row]

    # Dynamically generate the placeholders for the insert query
    insert_query = f"INSERT INTO [{table_name}] VALUES ({','.join(['?' for _ in cleaned_row])})"
    
    # Execute the insert query, inserting cleaned data
    cursor.execute(insert_query, tuple(cleaned_row))

# Commit after inserting all rows
conn.commit()

print(f"Data from {file_path} inserted into table [{table_name}].")

# Close the connection
conn.close()

Table 'trending' created successfully.
Data from C:\Users\khalil\marketing_data_Analytics_Project\youtube_trending_data.csv inserted into table [trending].
